In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

print("✓ Libraries imported successfully")

## 1. Завантаження та підготовка даних

In [ ]:
# Load data
control_df = pd.read_csv('control_group.csv', sep=';', encoding='utf-8')
test_df = pd.read_csv('test_group.csv', sep=';', encoding='utf-8')

# Convert dates
control_df['Date'] = pd.to_datetime(control_df['Date'], format='%d.%m.%Y')
test_df['Date'] = pd.to_datetime(test_df['Date'], format='%d.%m.%Y')

# Remove missing data (Aug 5 in control)
funnel_cols = ['# of Impressions', 'Reach', '# of Website Clicks', 
               '# of Searches', '# of View Content', '# of Add to Cart', '# of Purchase']

control_df = control_df.dropna(subset=funnel_cols, how='all')

print(f"Control data: {len(control_df)} days")
print(f"Test data: {len(test_df)} days")
print("\n✓ Data loaded and cleaned")

## 2. Розрахунок агрегованих метрик воронки

In [ ]:
# Calculate aggregate funnel metrics
def calculate_funnel(df, group_name):
    funnel = {
        'Group': group_name,
        'Impressions': df['# of Impressions'].sum(),
        'Clicks': df['# of Website Clicks'].sum(),
        'Searches': df['# of Searches'].sum(),
        'View Content': df['# of View Content'].sum(),
        'Add to Cart': df['# of Add to Cart'].sum(),
        'Purchase': df['# of Purchase'].sum(),
    }
    return funnel

control_funnel = calculate_funnel(control_df, 'Control')
test_funnel = calculate_funnel(test_df, 'Test')

# Create DataFrame
funnel_df = pd.DataFrame([control_funnel, test_funnel])

print("Funnel Metrics:")
print(funnel_df.to_string(index=False))
print("\n✓ Funnel metrics calculated")

## 3. Розрахунок коефіцієнтів конверсії між етапами

In [ ]:
# Calculate stage-to-stage conversion rates
stages = ['Impressions', 'Clicks', 'Searches', 'View Content', 'Add to Cart', 'Purchase']

def calculate_conversion_rates(row):
    rates = {}
    rates['Impr→Click (CTR)'] = (row['Clicks'] / row['Impressions'] * 100) if row['Impressions'] > 0 else 0
    rates['Click→Search'] = (row['Searches'] / row['Clicks'] * 100) if row['Clicks'] > 0 else 0
    rates['Search→View'] = (row['View Content'] / row['Searches'] * 100) if row['Searches'] > 0 else 0
    rates['View→Cart'] = (row['Add to Cart'] / row['View Content'] * 100) if row['View Content'] > 0 else 0
    rates['Cart→Purchase'] = (row['Purchase'] / row['Add to Cart'] * 100) if row['Add to Cart'] > 0 else 0
    rates['Overall (Click→Purchase)'] = (row['Purchase'] / row['Clicks'] * 100) if row['Clicks'] > 0 else 0
    return pd.Series(rates)

conversion_df = funnel_df.copy()
conversion_rates = funnel_df.apply(calculate_conversion_rates, axis=1)
conversion_df = pd.concat([conversion_df[['Group']], conversion_rates], axis=1)

print("\nConversion Rates (Stage-to-Stage):")
print(conversion_df.to_string(index=False))
print("\n✓ Conversion rates calculated")

## 4. Візуалізація #1: Класична воронка (Plotly)

In [ ]:
# Create funnel chart with Plotly
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Control Campaign', 'Test Campaign'),
    specs=[[{'type': 'funnel'}, {'type': 'funnel'}]],
    horizontal_spacing=0.15
)

# Control funnel
fig.add_trace(go.Funnel(
    name='Control',
    y=stages,
    x=[control_funnel[s] for s in stages],
    textposition="inside",
    textinfo="value+percent initial",
    marker={"color": ["#637FA8", "#7A92B8", "#91A5C8", "#A8B8D8", "#BFCBE8", "#D6DEF8"]},
    connector={"line": {"color": "#637FA8", "width": 2}}
), row=1, col=1)

# Test funnel
fig.add_trace(go.Funnel(
    name='Test',
    y=stages,
    x=[test_funnel[s] for s in stages],
    textposition="inside",
    textinfo="value+percent initial",
    marker={"color": ["#E87E7E", "#ED9191", "#F2A4A4", "#F7B7B7", "#FCCACA", "#FFDDDD"]},
    connector={"line": {"color": "#E87E7E", "width": 2}}
), row=1, col=2)

fig.update_layout(
    title_text="<b>Порівняння воронок продажів: Control vs Test</b>",
    title_font_size=20,
    height=600,
    showlegend=False,
    font=dict(size=12)
)

fig.show()
print("\n✓ Funnel chart created")

## 5. Візуалізація #2: Порівняльний bar chart

In [ ]:
# Prepare data for grouped bar chart
funnel_melted = funnel_df.melt(id_vars='Group', var_name='Stage', value_name='Count')
funnel_melted['Stage'] = pd.Categorical(funnel_melted['Stage'], categories=stages, ordered=True)
funnel_melted = funnel_melted.sort_values('Stage')

# Create grouped bar chart
fig = px.bar(
    funnel_melted,
    x='Stage',
    y='Count',
    color='Group',
    barmode='group',
    title='<b>Абсолютні значення на кожному етапі воронки</b>',
    labels={'Count': 'Кількість', 'Stage': 'Етап воронки'},
    color_discrete_map={'Control': '#637FA8', 'Test': '#E87E7E'},
    text='Count'
)

fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig.update_layout(
    height=500,
    xaxis_title='Етап воронки',
    yaxis_title='Кількість користувачів',
    font=dict(size=12),
    legend=dict(title='Група', orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig.show()
print("\n✓ Bar chart created")

## 6. Візуалізація #3: Коефіцієнти конверсії між етапами

In [ ]:
# Prepare conversion rates for visualization
conv_cols = ['Impr→Click (CTR)', 'Click→Search', 'Search→View', 'View→Cart', 'Cart→Purchase']
conversion_plot_df = conversion_df[['Group'] + conv_cols].melt(
    id_vars='Group', var_name='Transition', value_name='Conversion Rate (%)'
)

# Create grouped bar chart for conversion rates
fig = px.bar(
    conversion_plot_df,
    x='Transition',
    y='Conversion Rate (%)',
    color='Group',
    barmode='group',
    title='<b>Коефіцієнти конверсії між етапами воронки</b>',
    labels={'Transition': 'Перехід між етапами', 'Conversion Rate (%)': 'Конверсія (%)'},
    color_discrete_map={'Control': '#637FA8', 'Test': '#E87E7E'},
    text='Conversion Rate (%)'
)

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    height=500,
    xaxis_title='Перехід між етапами',
    yaxis_title='Конверсія (%)',
    font=dict(size=12),
    legend=dict(title='Група', orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig.show()
print("\n✓ Conversion rates chart created")

## 7. Візуалізація #4: Drop-off rates (Відсів на кожному етапі)

In [ ]:
# Calculate drop-off rates
def calculate_dropoff(row):
    dropoff = {}
    dropoff['Impr→Click'] = 100 - (row['Clicks'] / row['Impressions'] * 100) if row['Impressions'] > 0 else 0
    dropoff['Click→Search'] = 100 - (row['Searches'] / row['Clicks'] * 100) if row['Clicks'] > 0 else 0
    dropoff['Search→View'] = 100 - (row['View Content'] / row['Searches'] * 100) if row['Searches'] > 0 else 0
    dropoff['View→Cart'] = 100 - (row['Add to Cart'] / row['View Content'] * 100) if row['View Content'] > 0 else 0
    dropoff['Cart→Purchase'] = 100 - (row['Purchase'] / row['Add to Cart'] * 100) if row['Add to Cart'] > 0 else 0
    return pd.Series(dropoff)

dropoff_df = funnel_df.copy()
dropoff_rates = funnel_df.apply(calculate_dropoff, axis=1)
dropoff_df = pd.concat([dropoff_df[['Group']], dropoff_rates], axis=1)

# Prepare for visualization
dropoff_plot_df = dropoff_df.melt(
    id_vars='Group', var_name='Stage Transition', value_name='Drop-off Rate (%)'
)

# Create chart
fig = px.bar(
    dropoff_plot_df,
    x='Stage Transition',
    y='Drop-off Rate (%)',
    color='Group',
    barmode='group',
    title='<b>Drop-off rates (Відсів користувачів на кожному етапі)</b>',
    labels={'Stage Transition': 'Перехід', 'Drop-off Rate (%)': 'Відсів (%)'},
    color_discrete_map={'Control': '#637FA8', 'Test': '#E87E7E'},
    text='Drop-off Rate (%)'
)

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    height=500,
    xaxis_title='Перехід між етапами',
    yaxis_title='Drop-off Rate (%)',
    font=dict(size=12),
    legend=dict(title='Група', orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig.show()
print("\n✓ Drop-off rates chart created")

## 8. Візуалізація #5: Нормалізована воронка (% від початкового етапу)

In [ ]:
# Calculate normalized funnel (percentage from Impressions)
def normalize_funnel(row):
    base = row['Impressions']
    if base == 0:
        return pd.Series([0] * 6)
    return pd.Series([
        100,  # Impressions (baseline)
        row['Clicks'] / base * 100,
        row['Searches'] / base * 100,
        row['View Content'] / base * 100,
        row['Add to Cart'] / base * 100,
        row['Purchase'] / base * 100,
    ])

normalized_df = funnel_df.copy()
normalized_values = funnel_df.apply(normalize_funnel, axis=1)
normalized_values.columns = stages
normalized_df = pd.concat([normalized_df[['Group']], normalized_values], axis=1)

# Melt for plotting
normalized_melted = normalized_df.melt(id_vars='Group', var_name='Stage', value_name='Percentage')
normalized_melted['Stage'] = pd.Categorical(normalized_melted['Stage'], categories=stages, ordered=True)

# Create line chart
fig = px.line(
    normalized_melted,
    x='Stage',
    y='Percentage',
    color='Group',
    markers=True,
    title='<b>Нормалізована воронка (% від Impressions)</b>',
    labels={'Percentage': '% від початкового етапу', 'Stage': 'Етап воронки'},
    color_discrete_map={'Control': '#637FA8', 'Test': '#E87E7E'}
)

fig.update_traces(marker=dict(size=10), line=dict(width=3))
fig.update_layout(
    height=500,
    xaxis_title='Етап воронки',
    yaxis_title='Відсоток від Impressions (%)',
    font=dict(size=12),
    legend=dict(title='Група', orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    yaxis=dict(range=[0, 105])
)

fig.show()
print("\n✓ Normalized funnel chart created")

## 9. Порівняльна таблиця з ключовими метриками

In [ ]:
# Create comprehensive comparison table
comparison_data = []

for stage in stages:
    ctrl_val = control_funnel[stage]
    test_val = test_funnel[stage]
    diff = test_val - ctrl_val
    pct_change = (diff / ctrl_val * 100) if ctrl_val > 0 else 0
    
    comparison_data.append({
        'Етап': stage,
        'Control': f"{ctrl_val:,.0f}",
        'Test': f"{test_val:,.0f}",
        'Різниця': f"{diff:+,.0f}",
        'Зміна (%)': f"{pct_change:+.1f}%"
    })

comparison_table = pd.DataFrame(comparison_data)

print("\n" + "="*80)
print("ПОРІВНЯЛЬНА ТАБЛИЦЯ ВОРОНКИ")
print("="*80)
print(comparison_table.to_string(index=False))
print("="*80)

## 10. Висновки та інсайти

In [ ]:
print("\n" + "="*80)
print("КЛЮЧОВІ ВИСНОВКИ З АНАЛІЗУ ВОРОНКИ")
print("="*80)

print("\n1. СИЛЬНІ СТОРОНИ TEST КАМПАНІЇ:")
print("   ✅ CTR (Impr→Click): +66.5% — значно краща ефективність реклами")
print("   ✅ Cart→Purchase: +47.0% — вища якість трафіку, більше завершених покупок")
print("   ✅ Менше показів, але БІЛЬШЕ результату (кращий CTR компенсує)")

print("\n2. TRADE-OFFS (КОМПРОМІСИ):")
print("   ⚠️ Impressions: -29.6% — менший охват аудиторії")
print("   ⚠️ Add to Cart: -29.9% — менше додавань в кошик")
print("   ⚠️ Але це компенсується вищою конверсією Cart→Purchase!")

print("\n3. НЕЙТРАЛЬНІ МЕТРИКИ:")
print("   → Total Purchases: майже однаково (15,161 vs 15,637, +3.1%)")
print("   → View Content: схоже (-1.1%)")

print("\n4. КРИТИЧНІ ІНСАЙТИ:")
print("   🎯 Test кампанія працює ЕФЕКТИВНІШЕ:")
print("      - Менше waste (менше impressions на той самий результат)")
print("      - Кращий targeting або creative performance")
print("      - Вищий intent to purchase (Cart→Purchase +47%)")

print("\n5. ВУЗЬКІ МІСЦЯ ВОРОНКИ:")

# Calculate biggest drop-offs
ctrl_worst = dropoff_df[dropoff_df['Group'] == 'Control'].iloc[0, 1:].idxmax()
test_worst = dropoff_df[dropoff_df['Group'] == 'Test'].iloc[0, 1:].idxmax()

print(f"   Control: Найбільший drop-off на {ctrl_worst}")
print(f"   Test: Найбільший drop-off на {test_worst}")

print("\n6. РЕКОМЕНДАЦІЇ:")
print("   ✅ Запустити Test підхід на 100% — він ефективніший")
print("   🔍 Дослідити причини високого CTR (creative, targeting, placement)")
print("   📈 Масштабувати бюджет Test кампанії для більшого охоплення")
print("   🎨 Оптимізувати View→Cart перехід для обох груп (найбільший drop-off)")

print("\n" + "="*80)
print("ФІНАЛЬНИЙ ВЕРДИКТ: Test Campaign ВИГРАЄ за ефективністю!")
print("="*80 + "\n")

## 11. Експорт результатів

In [ ]:
# Export funnel data to Excel for further analysis
with pd.ExcelWriter('Fefelov_Final_Project-4_Funnel_Analysis.xlsx', engine='openpyxl') as writer:
    funnel_df.to_excel(writer, sheet_name='Funnel_Metrics', index=False)
    conversion_df.to_excel(writer, sheet_name='Conversion_Rates', index=False)
    dropoff_df.to_excel(writer, sheet_name='Dropoff_Rates', index=False)
    normalized_df.to_excel(writer, sheet_name='Normalized_Funnel', index=False)
    comparison_table.to_excel(writer, sheet_name='Comparison_Table', index=False)

print("✓ Results exported to 'Fefelov_Final_Project-4_Funnel_Analysis.xlsx'")
print("\n✅ Аналіз воронки завершено успішно!")

---

## Підсумок

Цей notebook містить:
- ✅ Класичні воронки для Control та Test груп
- ✅ Порівняльні bar charts абсолютних значень
- ✅ Візуалізацію коефіцієнтів конверсії між етапами
- ✅ Аналіз drop-off rates (відсіву)
- ✅ Нормалізовану воронку (% від початку)
- ✅ Порівняльні таблиці та інсайти

**Висновок:** Test Campaign демонструє значно кращу ефективність за ключовими метриками (CTR +66%, Cart→Purchase +47%) і рекомендується до впровадження.

---

**Автор:** Fefelov  
**Дата:** 6 листопада 2025 року  
**Курс:** GoIT Product Analytics - Final Project